In [1]:
import os
from utils.config import GRAPH_SAVE_FOLDER, DATA_FOLDER
from utils.plotter import plot_line_comparison


In [2]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"OneWeb_channel_specs_250MHz_10.5GHz"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
# df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [3]:
# import pandas as pd

# data_dir = os.path.join(DATA_FOLDER)
# extra_tag = f"OneWeb_BGAN"    
# tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# # Ensure the tagged folder exists
# os.makedirs(tagged_folder, exist_ok=True)

# df = pd.read_csv('Satellite_Australia_Simulation_Log.csv')
# # df = pd.read_csv('./250mhz_Data_10.5Ghz/Satellite_Australia_Simulation_Log.csv')

In [4]:
import pandas as pd

data_dir = os.path.join(DATA_FOLDER)
extra_tag = f"starlink"    
tagged_folder = os.path.join(GRAPH_SAVE_FOLDER, extra_tag)

# Ensure the tagged folder exists
os.makedirs(tagged_folder, exist_ok=True)

df = pd.read_csv('Satellite_Australia_Simulation_Log_starlink.csv')

In [5]:
import pandas as pd
import re

# Step 1: Drop columns that are all zeros or all NaN
df = df.drop(columns=[col for col in df.columns if ((df[col] == 0) | (df[col].isna())).all()])

# Step 2: Get max satellite ID from column names
sat_ids = [int(match.group(1)) for col in df.columns if (match := re.match(r'LEO(\d+)_', col))]
max_sat_id = max(sat_ids) if sat_ids else -1

print("Max satellite ID found:", max_sat_id)

# Step 3: Track satellites with access
possible_sat_conn_ids = []

for i in range(max_sat_id + 1):
    sat_id = f'LEO{i}'

    # Check access columns
    sydney_access_col = f'{sat_id}_Sydney_Access'
    melbourne_access_col = f'{sat_id}_Melbourne_Access'

    # Check if access columns exist and have any True values
    # .fillna(False) ensures we treat NaN as False
    # .any() checks if there is at least one True value
    sydney_access_ok = sydney_access_col in df.columns and df[sydney_access_col].fillna(False).any()
    melbourne_access_ok = melbourne_access_col in df.columns and df[melbourne_access_col].fillna(False).any()

    if sydney_access_ok or melbourne_access_ok:
        possible_sat_conn_ids.append(sat_id)
    else:
        # Drop base satellite info if no access
        base_cols = [f'{sat_id}_Name', f'{sat_id}_Lat', f'{sat_id}_Lon', f'{sat_id}_Freq_Hz']
        existing_cols_to_drop = [col for col in base_cols if col in df.columns]
        df = df.drop(columns=existing_cols_to_drop)

print("Satellites with possible connection:", possible_sat_conn_ids)
print("length(Satellites with possible connection):", len(possible_sat_conn_ids))


Max satellite ID found: 64
Satellites with possible connection: ['LEO16', 'LEO23', 'LEO30', 'LEO37']
length(Satellites with possible connection): 4


In [6]:
# Collect all satellite-related columns NOT in possible_sat_conn_ids to drop
cols_to_drop = []

for col in df.columns:
    # Extract satellite id number from column name if it matches pattern
    match = re.match(r'(LEO\d+)_', col)
    if match:
        sat_id = match.group(1)
        # If satellite not in possible_sat_conn_ids, mark its columns for deletion
        if sat_id not in possible_sat_conn_ids:
            cols_to_drop.append(col)

# Drop those columns
df = df.drop(columns=cols_to_drop)

print("Remaining columns after cleanup:", df.columns.tolist())


Remaining columns after cleanup: ['Time', 'LEO16_Name', 'LEO16_Lat', 'LEO16_Lon', 'LEO16_Freq_Hz', 'LEO16_Sydney_Access', 'LEO16_Sydney_SNR_dB', 'LEO16_Sydney_RSSI_dBm', 'LEO16_Sydney_Throughput', 'LEO16_Sydney_BER_QPSK', 'LEO16_Sydney_BER_MQAM', 'LEO16_Sydney_Latency', 'LEO16_Sydney_TimeOut', 'LEO16_Melbourne_Access', 'LEO16_Melbourne_SNR_dB', 'LEO16_Melbourne_RSSI_dBm', 'LEO16_Melbourne_Throughput', 'LEO16_Melbourne_BER_QPSK', 'LEO16_Melbourne_BER_MQAM', 'LEO16_Melbourne_Latency', 'LEO16_Melbourne_TimeOut', 'LEO23_Name', 'LEO23_Lat', 'LEO23_Lon', 'LEO23_Freq_Hz', 'LEO23_Sydney_Access', 'LEO23_Sydney_SNR_dB', 'LEO23_Sydney_RSSI_dBm', 'LEO23_Sydney_Throughput', 'LEO23_Sydney_BER_QPSK', 'LEO23_Sydney_BER_MQAM', 'LEO23_Sydney_Latency', 'LEO23_Sydney_TimeOut', 'LEO23_Melbourne_Access', 'LEO23_Melbourne_SNR_dB', 'LEO23_Melbourne_RSSI_dBm', 'LEO23_Melbourne_Throughput', 'LEO23_Melbourne_BER_QPSK', 'LEO23_Melbourne_BER_MQAM', 'LEO23_Melbourne_Latency', 'LEO23_Melbourne_TimeOut', 'LEO30_Name'

In [7]:
all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]

In [16]:
df.head(2)

,Time,LEO16_Name,LEO16_Lat,LEO16_Lon,LEO16_Freq_Hz,LEO16_Sydney_Access,LEO16_Sydney_SNR_dB,LEO16_Sydney_RSSI_dBm,LEO16_Sydney_Throughput,LEO16_Sydney_BER_QPSK,...,LEO37_Sydney_Latency,LEO37_Sydney_TimeOut,LEO37_Melbourne_Access,LEO37_Melbourne_SNR_dB,LEO37_Melbourne_RSSI_dBm,LEO37_Melbourne_Throughput,LEO37_Melbourne_BER_QPSK,LEO37_Melbourne_BER_MQAM,LEO37_Melbourne_Latency,LEO37_Melbourne_TimeOut
0,10-Apr-2025 12:00:00,Starlink-Shell-1_16,-30.461486,150.453208,12300000000,1,22.785880,-97.165211,1.894220e+09,6.690798e-85,...,0.006708,10-Apr-2025 12:00:00,1,-13.188454,-133.139545,1.690638e+07,0.378354,0.345734,0.007897,10-Apr-2025 12:00:00
1,10-Apr-2025 12:01:00,Starlink-Shell-1_16,-27.720286,153.174770,12500000000,1,17.552033,-102.399058,1.463947e+09,7.120663e-27,...,0.005487,10-Apr-2025 12:01:00,1,-2.949468,-122.900559,1.479325e+08,0.156961,0.281304,0.006895,10-Apr-2025 12:01:00


In [8]:
for i, row in df.iterrows():
    # Get the highest value across all RSSI columns at the `i-th` row
    highest_rssi_at_i = df[all_rssi_cols].iloc[i].max()

    # Get the column name that has the highest RSSI value at the `i-th` index
    column_with_highest_rssi = df[all_rssi_cols].iloc[i].idxmax()

    print(f"Highest RSSI value at index {i}: {highest_rssi_at_i}")
    print(f"Column with the highest RSSI value at index {i}: {column_with_highest_rssi}")

Highest RSSI value at index 0: -97.1652112047367
Column with the highest RSSI value at index 0: LEO16_Sydney_RSSI_dBm
Highest RSSI value at index 1: -102.399057626458
Column with the highest RSSI value at index 1: LEO16_Sydney_RSSI_dBm
Highest RSSI value at index 2: -105.297424809908
Column with the highest RSSI value at index 2: LEO37_Sydney_RSSI_dBm
Highest RSSI value at index 3: -105.136379076059
Column with the highest RSSI value at index 3: LEO37_Sydney_RSSI_dBm


In [9]:
stats = df[all_rssi_cols].fillna(0).describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

In [10]:
stats = df[all_rssi_cols].describe().T
stats.to_csv(os.path.join('./', 'rssi_stats.csv'))

In [11]:
stats = df[all_snr_cols].describe().T
stats.to_csv(os.path.join('./', 'snr_stats.csv'))

In [12]:

# Highest value per column
max_per_column = df[all_rssi_cols].max()

# Highest value among all columns
overall_max = df[all_rssi_cols].max().max()

# Displaying results
print("\nHighest Value per Column:\n", max_per_column)
print("\nHighest Value Among All Columns:", overall_max)


Highest Value per Column:
 LEO16_Sydney_RSSI_dBm       -97.165211
LEO16_Melbourne_RSSI_dBm   -102.173460
LEO23_Sydney_RSSI_dBm      -120.342141
LEO23_Melbourne_RSSI_dBm   -114.671985
LEO30_Sydney_RSSI_dBm      -115.905631
LEO30_Melbourne_RSSI_dBm   -103.378039
LEO37_Sydney_RSSI_dBm      -105.136379
LEO37_Melbourne_RSSI_dBm   -110.388725
dtype: float64

Highest Value Among All Columns: -97.1652112047367


In [13]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build SNR column names
    sydney_SNR_col = f'{sat_id}_Sydney_SNR_dB'
    melbourne_SNR_col = f'{sat_id}_Melbourne_SNR_dB'

    # Check which SNR columns exist
    sydney_exists = sydney_SNR_col in df.columns
    melbourne_exists = melbourne_SNR_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_SNR_col],
            labels=[sydney_SNR_col],
            xlabel='Time (m)',
            ylabel='dBm',
            title=f'SNR for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_SNR_.png",
            folder=tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_SNR_col],
            labels=[melbourne_SNR_col],
            xlabel='Time (m) ',
            ylabel='dBm',
            title=f'SNR for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_SNR__melbourne_.png",
            folder=tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [14]:
# Loop over each satellite in possible_sat_conn_ids
for sat_id in possible_sat_conn_ids:
    # Build RSSI column names
    sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
    melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

    # Check which RSSI columns exist
    sydney_exists = sydney_RSSI_col in df.columns
    melbourne_exists = melbourne_RSSI_col in df.columns

    # Skip if neither exists
    if not sydney_exists and not melbourne_exists:
        continue
    # Plot 
    if sydney_exists:
        plot_line_comparison(
            df,
            columns=[sydney_RSSI_col],
            labels=[sydney_RSSI_col],
            xlabel='Time (m)',
            ylabel='dBm',
            title=f'RSSI for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_Sydney_RSSI_.png",
            folder=tagged_folder
        )
    if melbourne_exists:
        plot_line_comparison(
            df,
            columns=[melbourne_RSSI_col],
            labels=[melbourne_RSSI_col],
            xlabel='Time (m) ',
            ylabel='dBm',
            title=f'RSSI for OneWeb Satellites',
            filename=f"{extra_tag}_{sat_id}_RSSI__melbourne_.png",
            folder=tagged_folder
        )

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [15]:
# import matplotlib.pyplot as plt
# import re

# all_rssi_cols = [col for col in df.columns if re.search(r'_RSSI_dBm$', col)]
# all_snr_cols = [col for col in df.columns if re.search(r'_SNR_dB$', col)]

# # Loop over each satellite in possible_sat_conn_ids
# for sat_id in possible_sat_conn_ids:
#     # Build RSSI column names
#     sydney_RSSI_col = f'{sat_id}_Sydney_RSSI_dBm'
#     melbourne_RSSI_col = f'{sat_id}_Melbourne_RSSI_dBm'

#     # Check which RSSI columns exist
#     sydney_exists = sydney_RSSI_col in df.columns
#     melbourne_exists = melbourne_RSSI_col in df.columns

#     # Skip if neither exists
#     if not sydney_exists and not melbourne_exists:
#         continue

#     plt.figure(figsize=(10, 5))
#     plt.title(f'RSSI vs Time for {sat_id}')
#     plt.xlabel('Time')
#     plt.ylabel('RSSI (dB)')

#     if sydney_exists:
#         plt.plot(df['Time'], df[sydney_RSSI_col], label='Sydney', color='blue')
#     if melbourne_exists:
#         plt.plot(df['Time'], df[melbourne_RSSI_col], label='Melbourne', color='green')

#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()
